### Downloading Dataset from Kaggle

In [ ]:
# !wget https://www.kaggle.com/snap/amazon-fine-food-reviews

### UnZip the archeive.zip in the current directory

In [ ]:
# import zipfile
# with zipfile.ZipFile("archive.zip","r") as zip_ref:
#     zip_ref.extractall()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/KNN Classifier on Amazon Food Reviews')

Mounted at /content/drive


# Amazon Fine Food Reviews Analysis

The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.

1. Number of reviews: 568,454
2. Number of users: 256,059
3. Number of products: 74,258
4. Timespan: Oct 1999 - Oct 2012
5. Number of Attributes/Columns in data: 10

Objective:
Given a review, determine whether the review is positive (Rating of 4 or 5) or Neutral(rating= 3)  or Critical (rating of 1 or 2).


We could use the Score/Rating. 
1. A rating of 4 or 5 could be considered a positive review.
2. A review of 1 or 2 could be considered Critical. 
3. A review of 3 is Nuetral 

In [ ]:
# Import libraries
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")



import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

# Installing a package
# import sys
# !{sys.executable} -m pip install [package_name]
# !{sys.executable} -m pip install gensim

# To get latest version of gensim
# !{sys.executable} -m pip install --upgrade gensim

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

## Reading the Data

In [ ]:
# using the SQLite Table to read data.
con = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/KNN Classifier on Amazon Food Reviews/database.sqlite')

filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews LIMIT 10000""", con) 

# Give reviews with Score>3 a positive rating, and reviews with a 
# score<3 a Critcical rating and reviews with a score = 3 Netrual.
def partition(x):
    if x < 3:
      return 0
    elif x == 3:
      return 1
    else:
      return 2

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
mapped_score = actualScore.map(partition) 
filtered_data['Score'] = mapped_score
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (10000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,2,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,2,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


#  Exploratory Data Analysis


## Data Cleaning: Deduplication

In [ ]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(9515, 10)

In [ ]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

95.15

In [ ]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive ,Neutral and Critical reviews are present in our dataset?
final['Score'].value_counts()

(9515, 10)


2    7285
0    1433
1     797
Name: Score, dtype: int64

> From above we can see that the dataset is highly unbalanced with

 1. Positive : 2 : 7285
 2. Neutral : 1 : 797
 3. Critical : 0 : 1433

In [ ]:
#  Just selecting the Text and Score column for further steps.
final = final[['Text','Score']]
final.head(5)

,Text,Score
2774,We have used the Victor fly bait for 3 seasons...,2
2773,Why is this $[...] when the same product is av...,2
1243,I just received my shipment and could hardly w...,2
1244,This was a really good idea and the final prod...,2
9524,If you're new to this product you need to be v...,2


## Upsampling the Minority Class data to create balanced dataset

**Note:**
### Upsampling should only be done if the Classifier/algorithm does not provide option of Class_weight
Class_weight is same as upsampling. Hence below code can be avoided by just providing class weights in algorithm. 
Note: KNN does not provide option for class weight hence we can use below upsampling code.

**To see how class weight works: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html**

### As we can see the dataset is highly Imbalanced, we can upsample the minority class dataset using sklearn ###
> Upsample Neutral datapoints and Critical datapoints to level of Positive Datapoints. 

In [ ]:
# Upsampling: Making # of datapoints of minority class equal to majority class
# slearns resample does Upsampling: 
# parameters: n_samples : Total # of samples to create

# Upsampling Neutral datapoints

from sklearn.utils import resample
#
# Create oversampled training data set for minority class
#
text_neutral__oversampled, y_neutral_oversampled = resample(final['Text'].loc[final['Score'] == 1],
                                        final['Score'].loc[final['Score'] == 1],
                                        replace=True,
                                        n_samples=(final['Text'].loc[final['Score'] == 2]).shape[0],
                                        random_state=123)

neutral_oversampled_df = pd.DataFrame()
neutral_oversampled_df['Text'] = text_neutral__oversampled
neutral_oversampled_df['Score'] =y_neutral_oversampled

# Upsampling Critical datapoints

text_Critical__oversampled, y_Critical_oversampled = resample(final['Text'].loc[final['Score'] == 0],
                                        final['Score'].loc[final['Score'] == 0],
                                        replace=True,
                                        n_samples=final['Text'].loc[final['Score'] == 2].shape[0],
                                        random_state=123)


critical_oversampled_df = pd.DataFrame()
critical_oversampled_df['Text'] = text_Critical__oversampled
critical_oversampled_df['Score'] =y_Critical_oversampled
# critical_oversampled_df['Score'].head()

# Just selecting rows where class = 2 (Positive)
positive_df = final.loc[final['Score'] == 2]

# Creating final balanced dataframe
balanced_final = pd.concat([positive_df, neutral_oversampled_df,critical_oversampled_df ], ignore_index=True)
balanced_final['Score'].value_counts()


2    7285
1    7285
0    7285
Name: Score, dtype: int64

## Data Pre Processing

In [ ]:

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])


from bs4 import BeautifulSoup
import re
from tqdm import tqdm

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def clean_text(document):
    sentance = re.sub(r"http\S+", "", document)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
   
    # Remove any emojis in document
    result = re.sub(r'<span.*?/span>', '', sentance)
    result = re.sub(r'<a.*?/a>', '', result)
    result = re.sub(r'<p.*?/p>', '', result)
    sentence_without_tags = re.split(r'<.*?>', result)
    sent = ''
    for element in sentence_without_tags:
        sent += ' ' + str(element)

    # Removing Special Characters
    text = re.sub('[^a-zA-Z0-9\s]', '', sent)
    text = re.sub(r'(https|http)?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    # Remove new line characters
    text = re.sub(r'^\s*$', '', text)
    # Remove distracting single quotes
    text = re.sub("\'", "", text)
    # remove punctuation marks and other symbols
    text = re.sub('[^\w_\s-]', ' ', text)  
    
    # Removing stopwords
    text = ' '.join(e.lower() for e in text.split() if e.lower() not in stopwords)
    cleaned_article = text.strip()

    # tokens = nltk.word_tokenize(document)  # Tokenize sentences

    # can be ignored for Word2Vec
    # cleaned_article = ' '.join([stemmer.stem(item) for item in tokens])  # Stemming each token

    return cleaned_article


# reviews is a list of all reviews of user
reviews = balanced_final['Text'].tolist()

# preprocessed_reviews is a list of all clean reviews after preprocessing
preprocessed_reviews = list(map(clean_text, reviews))


In [ ]:
# Saving the Preproceed Text into a file so that it can be used further

preprocessed_reviews_df = pd.DataFrame()
preprocessed_reviews_df['Text'] = balanced_final['Text']
preprocessed_reviews_df['Processed Text'] = preprocessed_reviews
preprocessed_reviews_df['Score'] = balanced_final['Score']
# preprocessed_reviews_df.to_csv("preprocessed_reviews_df.csv")

In [ ]:
preprocessed_reviews_df.head(5)

,0,Score
0,used victor fly bait seasons ca not beat great...,2
1,product available victor traps unreal course t...,2
2,received shipment could hardly wait try produc...,2
3,really good idea final product outstanding use...,2
4,new product need careful dosage strong batches...,2
